In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 2. Cargar una imagen en escala de grises

img = cv2.imread('../datos/image.jpg', cv2.IMREAD_GRAYSCALE)
plt.imshow(img, cmap='gray')
plt.title('Imagen en escala de grises')
plt.axis('off')
plt.show()


In [ ]:
# 3. Aplicar umbral fijo
_, thresh_fijo = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
plt.imshow(thresh_fijo, cmap='gray')
plt.title('Umbral fijo')
plt.axis('off')
plt.show()


In [ ]:
# 4. Aplicar umbral adaptativo
thresh_adapt = cv2.adaptiveThreshold(
    img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY, 11, 2
)
plt.imshow(thresh_adapt, cmap='gray')
plt.title('Umbral adaptativo')
plt.axis('off')
plt.show()


In [ ]:
# 5. Detectar contornos
contours, _ = cv2.findContours(thresh_adapt, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#6. Dibujar contornos sobre la imagen original (en color para visualización)
img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
cv2.drawContours(img_color, contours, -1, (0, 255, 0), 2)
plt.imshow(cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB))
plt.title('Contornos detectados')
plt.axis('off')
plt.show()


In [ ]:
#7. Calcular centro de masa y bounding boxes
for cnt in contours:
    M = cv2.moments(cnt)
    if M['m00'] != 0:
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        cv2.circle(img_color, (cx, cy), 5, (0, 0, 255), -1)

    x, y, w, h = cv2.boundingRect(cnt)
    cv2.rectangle(img_color, (x, y), (x + w, y + h), (255, 0, 0), 2)

plt.imshow(cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB))
plt.title('Centros de masa y bounding boxes')
plt.axis('off')
plt.show()

# Calcular métricas básicas
areas = [cv2.contourArea(c) for c in contours]
perimetros = [cv2.arcLength(c, True) for c in contours]

print("Número de formas detectadas:", len(contours))
print("Área promedio:", np.mean(areas))
print("Perímetro promedio:", np.mean(perimetros))



In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                   cv2.THRESH_BINARY, 11, 2)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
    cv2.imshow('Webcam - segmentación', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
